In [2]:
import os
import psycopg2

In [30]:
db_username = 'postgres'
db_password = '11111111'
os.system(
    f"""docker run --name postgresql -e POSTGRES_USER={db_username} -e POSTGRES_PASSWORD={db_password} -p 5432:5432 -d postgres"""
)

0

## Создаём таблицы

In [42]:
dsn = {
    'user': 'postgres',
    'password': '11111111'
}

In [254]:
create_tables = """
CREATE TABLE IF NOT EXISTS review (
    id INTEGER GENERATED ALWAYS AS IDENTITY,
    user_uuid UUID,
    movie_uuid UUID NULL,
    text_review TEXT,
    first_name varchar(50),
    last_name varchar(50),
    creation_date DATE,
    UNIQUE (user_uuid, movie_uuid)
);
CREATE TABLE IF NOT EXISTS rating (
    id UUID PRIMARY KEY,
    user_uuid UUID,
    movie_uuid UUID NULL,
    review_id INTEGER NULL,
    rating INTEGER,
    UNIQUE (user_uuid, movie_uuid),
    UNIQUE(user_uuid, review_id)
);
CREATE TABLE IF NOT EXISTS bookmark (
    id UUID PRIMARY KEY,
    user_uuid UUID,
    movie_uuid UUID NULL,
    UNIQUE (user_uuid, movie_uuid)
);
"""

In [256]:
connection = psycopg2.connect(**dsn)
cursor = conn.cursor()
cursor.execute(create_tables)

## Заполняем таблицы данными

In [257]:
from uuid import uuid4
import random
from datetime import date

In [258]:
movie_uuids = [str(uuid4()) for _ in range(20000)]
user_uuids = [str(uuid4()) for _ in range(20000)]

In [259]:
rating_movies = [
    (
        (uuid4(), random.choice(user_uuids), random.choice(movie_uuids)) for _ in range(1000)) for _ in range(1000)
]

rating_reviews = [
    (
        (uuid4(), random.choice(user_uuids), random.choice(list(range(1_000_000)))) for _ in range(1000)
    ) for _ in range(1000)
]

In [260]:
review_data = list()
data = list()
for movie in movie_uuids:
    data += [
        (random.choice(user_uuids), movie, 'x' * random.randint(100, 10000), 'test', 'user', str(date.today())) for _ in range(random.randint(5, 100))
    ]
    if len(data) >= 1000:
        review_data.append(data)
        data = list()

In [261]:
%%time
insert_str = "%s," * 6
insert_str = insert_str[:-1]
for data in review_data:
    cursor.executemany(f"""
    INSERT INTO review(user_uuid, movie_uuid, text_review, first_name, last_name, creation_date)
    VALUES({insert_str})
    ON CONFLICT(user_uuid, movie_uuid) DO NOTHING;""", data)
    

CPU times: total: 2min 17s
Wall time: 1h 25min 26s


In [262]:
cursor.execute('select count(*) from review')
reviews_count = cursor.fetchone()[0]
print(reviews_count)

1046476


In [263]:
rating_movies = [
    (
    (str(uuid4()), random.choice(user_uuids), random.choice(movie_uuids), random.randint(0, 10)) for _ in range(1000)
    ) for i in range(1000)
]
rating_reviews = [
    (
    (str(uuid4()), random.choice(user_uuids), random.randint(1, reviews_count), random.randint(0, 10)) for _ in range(1000)
    ) for i in range(1000)
]

In [264]:
%%time
insert_str = "%s," * 4
insert_str = insert_str[:-1]
for data in rating_movies:
    cursor.executemany(f"""
    INSERT INTO rating(id, user_uuid, movie_uuid, rating)
    VALUES({insert_str})
    ON CONFLICT(user_uuid, movie_uuid) DO NOTHING;""", data)

for data in rating_reviews:
    cursor.executemany(f"""
    INSERT INTO rating(id, user_uuid, review_id, rating)
    VALUES({insert_str})
    ON CONFLICT(user_uuid, review_id) DO NOTHING;""", data)
    

CPU times: total: 4min 25s
Wall time: 1h 42min 26s


In [265]:
cursor.execute('select count(*) from rating')
cursor.fetchall()

[(1998724,)]

In [266]:
bookmark_data = list()
data = list()
for user in user_uuids:
    data += [
        (str(uuid4()), user, random.choice(movie_uuids)) for _ in range(random.randint(5, 100))
    ]
    if len(data) >= 1000:
        bookmark_data.append(data)
        data = list()

In [267]:
%%time
insert_str = "%s," * 3
insert_str = insert_str[:-1]
for data in bookmark_data:
    cursor.executemany(f"""
    INSERT INTO bookmark(id, user_uuid, movie_uuid)
    VALUES({insert_str})
    ON CONFLICT(user_uuid, movie_uuid) DO NOTHING;""", data)


CPU times: total: 1min 31s
Wall time: 52min 2s


In [268]:
cursor.execute('select count(*) from bookmark')
cursor.fetchall()

[(1049098,)]

## Тестируем скорость получения данных

In [269]:
cursor.execute('SELECT movie_uuid from rating GROUP BY movie_uuid ORDER BY COUNT(movie_uuid) DESC LIMIT 1;')
movie = cursor.fetchall()[0][0]
cursor.execute('SELECT user_uuid from rating GROUP BY user_uuid ORDER BY COUNT(user_uuid) DESC LIMIT 1;')
user = cursor.fetchall()[0][0]

In [270]:
%%time
cursor.execute('SELECT movie_uuid FROM rating WHERE movie_uuid IS NOT NULL AND rating = 10;')
len(cursor.fetchall())

CPU times: total: 46.9 ms
Wall time: 107 ms


90743

In [271]:
%%time
cursor.execute(f"""
SElECT COUNT(id) as likes 
    FROM rating 
    WHERE movie_uuid = '{movie}' AND rating = 10;"""
              )
print('likes:', len(cursor.fetchall()))
cursor.execute(f"""
SElECT COUNT(id) as dislikes 
    FROM rating 
    WHERE movie_uuid = '{movie}' AND rating = 0;"""
              )
print('dislikes:', len(cursor.fetchall()))

likes: 1
dislikes: 1
CPU times: total: 0 ns
Wall time: 120 ms


In [272]:
%%time
cursor.execute(f"SELECT movie_uuid FROM bookmark WHERE user_uuid = '{user}';")
len(cursor.fetchall())


CPU times: total: 0 ns
Wall time: 2 ms


32

In [273]:
%%time
cursor.execute(f"SELECT AVG(rating) as avg_rating FROM rating WHERE movie_uuid = '{movie}';")
cursor.fetchall()

CPU times: total: 0 ns
Wall time: 55.3 ms


[(Decimal('5.1645569620253165'),)]